# 要約 
このJupyterノートブックは、Kaggleのコンペティション「LLM 20 Questions」に参加するためのAIエージェントを作成するプロセスを示しています。本ノートブックでは、質問者（質問をする側）と回答者（はいまたはいいえで回答する側）の2つのタイプのエージェントが実装されています。

### 問題に取り組む内容
ノートブックは、言語モデルを活用して「20の質問」ゲームを効果的にプレイできるエージェントを構築することを目的としています。プレイヤーは、はいまたはいいえで回答できる質問を通じて、特定のキーワードを推測する必要があります。このゲームにおいて、エージェントは一連の質問を経て、ターゲットワードを特定する能力を評価されます。

### 使用されている手法とライブラリ
1. **Gemmaモデル**: 本ノートブックでは、Googleが開発したGemmaという言語モデルが使用されています。特に、`gemma_pytorch`リポジトリからモデルがクローンされ、その構成と重みが利用されます。
2. **PyTorch**: モデルはPyTorchを用いて実装され、GPUによる計算が可能です。
3. **サブミッション生成**: 最終的に、エージェントは`submission.tar.gz`というファイルにまとめられ、Kaggleに提出される形式になっています。

### 構成
- ノートブックには、ライブラリのインストール手順、エージェントの初期化、プロンプトのフォーマット、質問者エージェント（`GemmaQuestionerAgent`）と回答者エージェント（`GemmaAnswererAgent`）のそれぞれのクラス定義が含まれています。
- 質問と応答のインタラクションを管理するためのロジックが実装されています。

### コード実行の手順
ノートブックの実行により、必要なライブラリやモデルがダウンロードされ、構築されたエージェントがKaggleのシミュレーション環境内で機能します。最後に、生成されたサブミッションファイルをKaggleに提出するための圧縮処理が行われます。

このように、このJupyterノートブックは、20の質問ゲームに基づいた言語モデルのエージェントを構築・提出するための包括的な手順を提供しています。

---


# 用語概説 
以下は、Jupyter Notebookの内容に関連する専門用語の簡単な解説です。初心者がつまずきそうなマイナーな用語や実務経験がないと馴染みのないものに焦点を当てました。

1. **ImmutableDict**:
   - 変更不可な辞書型のデータ構造。通常の辞書は要素の追加や削除が可能ですが、ImmutableDictは生成後にその要素を変更できません。この特性によって、データの整合性を保証できます。

2. **sentencepiece**:
   - 自然言語処理のためのテキストのトークン化ライブラリ。文をサブワード（通常、単語未満の単位）に分割し、言語モデルの効率を向上させます。特に、低リソース言語や新しい単語を扱う際に効果的です。

3. **CausalLM**:
   - 観察された情報から次に来る情報を予測するモデルの一種です。典型的な用途には言語生成があり、次の単語を予測しながら文を形成します。「CAUSAL」は時間的な因果関係に重きを置いています。

4. **デフォルトのtorch dtype**:
   - PyTorchにおけるデフォルトのデータ型設定。Tensor（多次元配列）の計算精度を指定し、float32やfloat64などのデータ型を使って数値演算が行えます。この設定を変更することで、計算効率やメモリ使用量に影響を与えることができます。

5. **Few-shot examples**:
   - 転移学習において、少数の例（トレーニングデータ）でモデルの性能を向上させる技術。例えば、質問の例とその応答を与えることで、モデルが新しい質問形式に適応する能力を高めます。

6. **interleave_unequal**:
   - 異なる長さの2つのリストを交互に組み合わせる処理を行う関数。一方のリストに要素が残っている場合、まだ要素を追加し続けることができます。このメソッドは、質問と回答などのペアを管理するのに有用です。

7. **sampler_kwargs**:
   - モデル生成プロセスにおいて、サンプリング方法を指定するための引数。生成時のランダム性を制御するために、温度（生成の多様性）、top_k（考慮する上位k候補）、top_p（確率のカットオフ）などの設定が含まれます。

8. **キーワード抽出**:
   - レスポンスから特定の情報を識別する処理。正確な単語やフレーズを見つけるために正規表現などを用いる手法が使われ、自然言語処理でよく見られます。

これらの用語は、機械学習や深層学習の文脈で特に特有の意味を持っており、初心者には理解が難しい場合がありますが、専門用語を知っておくことで理解が深まるでしょう。

---


このノートブックは、**LLM 20 Questions**のエージェント作成プロセスを示しています。このノートブックを実行すると、`submission.tar.gz`ファイルが生成されます。コンペティションの右側にある**Submit to competition**見出しから直接このファイルを提出できます。あるいは、ノートブックビューアから*Output*タブをクリックし、`submission.tar.gz`を見つけてダウンロードしてください。コンペティションのホームページの左上にある**Submit Agent**をクリックして、ファイルをアップロードし、提出を完了させてください。

In [ ]:
%%bash
# 作業ディレクトリに移動します
cd /kaggle/working 

# immutabledict と sentencepiece パッケージをインストールします
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece 

# gemma_pytorchリポジトリをGitHubからクローンします
git clone https://github.com/google/gemma_pytorch.git > /dev/null 

# gemmaフォルダを作成します
mkdir /kaggle/working/submission/lib/gemma/ 

# gemma_pytorchからのファイルを新しく作成したgemmaフォルダに移動します
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

In [ ]:
%%writefile submission/main.py
# セットアップ
import os
import sys

# **重要:** ノートブックとシミュレーション環境の両方でコードが動作するように、
# システムパスをこのように設定します。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from pathlib import Path

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

if os.path.exists(KAGGLE_AGENT_PATH):
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"

# プロンプトのフォーマット
import itertools
from typing import Iterable


class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    def __repr__(self):
        return self._state

    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self

    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self

    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self

    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self

    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self

    def reset(self):
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self


# エージェントの定義
import re


@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """指定されたデータ型にデフォルトのtorch dtypeを設定します。"""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)


class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        print("モデルの初期化中")
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant

        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt')
            model.load_weights(ckpt_path)
            self.model = model.to(self._device).eval()

    def __call__(self, obs, *args):
        self._start_session(obs)  # セッションを開始
        prompt = str(self.formatter)  # フォーマッタの状態を文字列に変換
        response = self._call_llm(prompt)  # LLMを呼び出して応答を取得
        response = self._parse_response(response, obs)  # 応答を解析
        print(f"{response=}")
        return response

    def _start_session(self, obs: dict):
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,  # 温度設定
                'top_p': 0.1,  # 確率のカットオフ
                'top_k': 1,  # トップkの設定
        }
        response = self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )
        return response

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)  # キーワードを抽出
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower()
        return keyword

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError


def interleave_unequal(x, y):
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]


class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()  # フォーマッタをリセット
        self.formatter.user("20の質問をプレイしましょう。あなたは質問者の役割を果たします。")
        turns = interleave_unequal(obs.questions, obs.answers)  # 質問と回答を交互に取得
        self.formatter.apply_turns(turns, start_agent='model')  # フォーマッタにターンを適用
        if obs.turnType == 'ask':
            self.formatter.user("はいかいいえで答えられる質問をしてください。")
        elif obs.turnType == 'guess':
            self.formatter.user("今、キーワードを推測してください。推測はダブルアスタリスクで囲んでください。")
        self.formatter.start_model_turn()  # モデルのターンを開始

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))  # 質問を抽出
            if match is None:
                question = "人ですか？"
            else:
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)  # 推測を解析
            return guess
        else:
            raise ValueError("不明なターンタイプ:", obs.turnType)


class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()  # フォーマッタをリセット
        self.formatter.user(f"20の質問をプレイしましょう。あなたは回答者の役割を果たします。キーワードは {obs.keyword} で、カテゴリーは {obs.category} です。")
        turns = interleave_unequal(obs.questions, obs.answers)  # 質問と回答を交互に取得
        self.formatter.apply_turns(turns, start_agent='user')  # フォーマッタにターンを適用
        self.formatter.user(f"質問はキーワード {obs.keyword} に関するものです。はいかいいえで答え、答えはダブルアスタリスクで囲んでください。")
        self.formatter.start_model_turn()  # モデルのターンを開始

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)  # 応答から答えを解析
        return 'yes' if 'yes' in answer else 'no'


# エージェントの作成
system_prompt = "あなたは20の質問ゲームをプレイするために設計されたAIアシスタントです。このゲームでは、答える側がキーワードを考え、質問する側がはいまたはいいえの質問に答えます。キーワードは特定の人、場所、または物です。"

few_shot_examples = [
    "20の質問をプレイしましょう。あなたは質問者の役割を果たします。最初の質問をしてください。",
    "人ですか？", "**いいえ**",
    "場所ですか？", "**はい**",
    "国ですか？", "**はい** 今、キーワードを推測してください。",
    "**フランス**", "正解です！",
]


# **重要:** エージェントをグローバルとして定義します。これにより、
# 必要なエージェントだけをロードできます。両方をロードすると、OOMが発生する可能性があります。
agent = None


def get_agent(name: str):
    global agent
    
    if agent is None and name == 'questioner':
        agent = GemmaQuestionerAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    elif agent is None and name == 'answerer':
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    assert agent is not None, "エージェントが初期化されていません。"

    return agent


def agent_fn(obs, cfg):
    if obs.turnType == "ask":
        response = get_agent('questioner')(obs)  # 質問者エージェントを呼び出す
    elif obs.turnType == "guess":
        response = get_agent('questioner')(obs)  # 質問者エージェントを呼び出す
    elif obs.turnType == "answer":
        response = get_agent('answerer')(obs)  # 回答者エージェントを呼び出す
    if response is None or len(response) <= 1:
        return "はい"  # デフォルト応答
    else:
        return response  # 応答を返す

In [ ]:
!apt install pigz pv > /dev/null  # pigzとpvをインストールします

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2  
# pigzを使用して、submissionフォルダの内容を圧縮し、同時にpvを使って進行状況を表示しながら、
# submission.tar.gzとして保存します。 
# また、gemma/pytorch/7b-it-quant/2からもファイルを追加します。

---

# コメント 

> ## Samar Elhissi
> 
> 例をありがとう。ローカルでのテスト方法は？
> 
> 
> 
> > ## Valentin Baltazar
> > 
> > ハードウェアがあるか確認してください…これらのLLMは多くの計算を必要とし、トレーニングとファインチューニングには強力なGPUが必要です。クラウドを使用する方がずっと簡単です。
> > 
> > 

---

> ## Michael Kamal 92
> 
> ありがとう。few_shot_examplesについてお尋ねしたいのですが、私は次のようにしなければなりませんか。
> 
> これのように    ('これは場所ですか？', 'はいまたはいいえ')
> 
> またはこれ    ('これは場所ですか？', 'はい',)
> 
> またはこれ     ('これは場所ですか？', 'はい', '今、キーワードを推測してください')
> 
> またはこれ     ('これは場所ですか？', 'いいえ', '今、キーワードを推測してください', 'フランス')
> 
> またはこれ     ('これは場所ですか？', 'はい', 'フランス')
> 
> どれが質問、回答、推測を作るのに正しいですか？
> 
> もう1つの質問は、Gemmaはfew_shot_examplesでトレーニングされていますか？

---

> ## Yukky_2801
> 
> こんにちは、私はKaggleの初心者です。あなたのノートブックを実行したとき、
> 
> 次のエラーが出ました：
> 
> tar: gemma/pytorch/7b-it-quant/2: Cannot stat: No such file or directory
> 
> 1.37MiB 0:00:00 [36.4MiB/s] [<=> ]
> 
> tar: 上記のエラーにより、終了ステータスが失敗になりました。
> 
> エラーが発生しているため、submission.tar.gzを提出できません。
> 
> これについてのアイデアはありません。解決策を提供してください。

> 
> > ## Andres H. Zapke
> > 
> > 明らかに、あなたはこのパス "gemma/pytorch/7b-it-quant/2" にアクセスしようとしています。そのパスにファイルがあるか確認してください（ノートブックの右側を見て、gemmaモデルがそこにあるか確認し、パスが一致するかを確認してください）。
> > 
> > > ## Aryan Singh
> > > 
> > > Add Input 機能を使用して、Gemma 7b-it-quant V2 を追加してください。
> > > 
> > > まず、こちらでライセンスを受け入れてください: [https://www.kaggle.com/models/google/gemma](https://www.kaggle.com/models/google/gemma)
> > > 
> > > 
> > > > ## Talal Mufti
> > > > 
> > > > パスにすべてのファイルがあったことを確認した後でも問題が発生したため、bashコマンドを少し編集しました。個人的には、これがよりうまくいきました：
> > > > 
> > > > !tar --use-compress-program='pigz --fast --recursive | pv' -f submission.tar.gz -c /kaggle/working/submission . -c /kaggle/input/gemma/pytorch/7b-it-quant/2
> > > > 

---

> ## Muhammad Hadi13
> 
> なぜファイルをコピーして実行しても、1.35 MB以上の出力が生成されないのか分かりません。これは常に検証エピソードで失敗します。Ryanの出力は約7GBでした。助けてください！！！

> 
> > ## Muhammad Hadi13
> > 
> > tar: gemma/pytorch: Cannot stat: No such file or directory
> > 
> > 1.37MiB 0:00:00 [36.4MiB/s] [<=>                                               ]
> > 
> > tar: 上記のエラーにより、終了ステータスが失敗になりました。
> > 
> > このエラーは提出セルブロックで発生します。
> > 
> > > ## Aryan Singh
> > > 
> > > まず、Gemma 7b-it-quant V2 を追加する必要があります。 
> > > 
> > > ノートブックでモデルを追加する機能を使用してください。 
> > > 
> > > まず、こちらでライセンスを受け入れてください: [https://www.kaggle.com/models/google/gemma](https://www.kaggle.com/models/google/gemma)
> > > 

---

> ## Ship of Theseus
> 
> ありがとうRyan、素晴らしい仕事です！ローカルで実行するための素晴らしいコードで、Kaggleコミュニティに共有してくれてありがとう。

---

> ## shiv_314
> 
> こんにちは！ 皆さん、1つ助けが必要です。gemmaパッケージでインポートエラーが発生しています。
> 
> 正しいシステムパスをPythonに設定することはできましたが、いまだに同じ問題が発生しています。助けてください！

---

> ## dedq
> 
> ありがとうRyan、素晴らしい仕事です！ローカルで実行するための素晴らしいコードで、Kaggleコミュニティに共有してくれてありがとう。

---

> ## Code Hacker
> 
> このノートブックの出力ファイルtar.gzを提出しようとしましたが、失敗しました…

> 
> > ## Code Hacker
> > 
> > モデルに同意していませんでした。以下の赤いボタンをクリックしてください…
> > 
> > 

---

> ## JAPerez
> 
> 素晴らしい仕事です、Ryan！

---

> ## philipha2
> 
> こんにちは、私はこのコンペティションの初心者です。 
> 
> あなたのノートブックを実行して提出しようとしたのですが、
> 
> submission.tar.gzファイルはどこに置けばよいですか？ 
> 
> Submit Agentボタンをクリックした後、このファイルを提出するだけでよいでしょうか？ 
> 
> 少し時間がかかります。
> 
> 私の質問は少し基本的に聞こえるかもしれませんが、返信してくれたら本当に感謝します！ 

> 
> > ## Kanchan Maurya
> > > Submit agentsをクリックした後にファイルを提出することでうまくいきます。最初のシミュレーションが動作しているため、時間がかかります。
> > > 

---

> ## vj4science
> 
> ありがとうRyan - これはコンペティションへの良いスタートです！とても感謝しています！

---

> ## gb_kwon
> 
> すごくクールなガイドラインを本当にありがとう！

---

> ## Andres H. Zapke
> 
> main.py内で、次のようにgemma_pytorchライブラリをインポートしています：from gemma.config.
> 
> これが私には機能しませんが、gemmaをインポートするとエラーはありません。
> 
> ローカルのgemmaモジュールのパスを手動で指定したり、Pythonライブラリ名でインポートしたりしました。何かアイデアはありますか？

---

> ## Duy Thai
> 
> こんにちは[@ryanholbrook](https://www.kaggle.com/ryanholbrook)、あなたのノートブックを試しましたが、"An attached model requires additional steps to be accessed. See the Models panel for details."というメッセージが表示されました。どうすれば良いでしょうか？
> 
> パネルを開くと、次のように表示されます：

> > ## Andres H. Zapke
> > > "Models"のメニューに行き、Gemmaを検索し、モデルのライセンスを受け入れてください。
> > > 

> > > > ## Duy Thai
> > > > ありがとう！
> > > > 

---

> ## Kai_Huang
> 
> こんにちは、Kaggleの初心者です。あなたのコードブロックを実行しようとした際に、以下のエラーが表示されました：
> 
> tar: gemma/pytorch/7b-it-quant/2: Cannot stat: No such file or directory
> 
> 1.37MiB 0:00:00 [36.4MiB/s] [<=>                                               ]
> 
> tar: 上記のエラーにより、終了ステータスが失敗になりました。
> 
> これについてのアイデアはありません。ありがとうございます！
> 
> > ## Kai_Huang
> > > ああ、わかりました。ノートブックにモデルを入力していなかった😱
> > > 

> > > > ## D Prince Armand KOUMI
> > > > モデルがなさすぎるので、1つ追加しようとしてみてください。
> > > > 

---

> ## Qusay AL-Btoush
> 
> すごく良いです、Ryan 

---